In [2]:
def refresh():
    mydb = pymysql.connect( host = str(entry1.get()) ,  user =str(entry2.get()) ,  passwd = str(entry3.get()), db = str(entry4.get())) 
    cursor = mydb.cursor()
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        q = "DROP TABLE IF EXISTS "+str(entry5.get())
        cursor.execute(q)
        label1 = tk.Label(root, text= 'Table Deleted if exists. Create the Table again!',font=('helvetica', 10))
        canvas1.create_window(600, 630, window=label1)
        global init_table
        init_table = False
    cursor.close()
    mydb.commit()
    mydb.close()
    

In [3]:
def create_database():
    mydb = pymysql.connect( host = str(entry1.get()) ,  user =str(entry2.get()) ,  passwd = str(entry3.get())) 
    cursor = mydb.cursor()
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        q = "CREATE DATABASE IF NOT EXISTS "+str(entry4.get())
        cursor.execute(q)
        label1 = tk.Label(root, text= 'Database named ' + str(entry4.get()) + ' created if not exists!',font=('helvetica', 10))
        canvas1.create_window(500, 210, window=label1)
    cursor.close()
    mydb.commit()
    mydb.close()
        

In [4]:
def create_table():
    mydb = pymysql.connect( host = str(entry1.get()) ,  user =str(entry2.get()) ,  passwd = str(entry3.get()) , db = str(entry4.get()))  
    cursor = mydb.cursor() 
    stmt = "SHOW TABLES LIKE '"+str(entry5.get())+"'"
    cursor.execute(stmt)
    result = cursor.fetchone()
    if result:
        label2 = tk.Label(root, text= 'Table named ' + str(entry5.get()) + ' already exists!',font=('helvetica', 10))
        canvas1.create_window(700, 210, window=label2)
    else:
        q = """CREATE TABLE IF NOT EXISTS """+ str(entry5.get())+"""(
       Name varchar(32) NOT NULL,
       Intro varchar(255) NOT NULL,
        Location varchar(32) NOT NULL,
        Job varchar(255) NOT NULL,
        About varchar(5000) NOT NULL,
        Education varchar(5000) NOT NULL,
        Skills varchar(5000) NOT NULL,
        URL varchar(255) NOT NULL
    )"""
        cursor.execute(q)
        label2 = tk.Label(root, text= 'Table named ' + str(entry5.get()) + ' does not exist! Table Created...',font=('helvetica', 10))
        canvas1.create_window(700, 210, window=label2)
    cursor.close()
    mydb.commit()
    mydb.close()

In [5]:
def Load_Database():
    import os
    global path
    file = askopenfile(mode ='r', filetypes =[('Excel Files', '*.xlsx *.csv')]) 
    s = str(file)[25:-29]
    path = os.path.abspath(s)
    entry6.insert(tk.END, s)
    

In [6]:
def update_database():
    import os
    global path
    global update_file
    import pandas as pd
    path = path.replace('\\','/')
    len1 = len(pd.read_excel(str(path)))
    os.startfile(str(path))
    if(len(pd.read_excel(str(path))) != len1):
        update_file = True
    

In [7]:
def initialize_table():
    #Initialize the Table
    import pymysql
    import xlrd
    mydb = pymysql.connect( host = str(entry1.get()) ,  user =str(entry2.get()) ,  passwd = str(entry3.get()) , db = str(entry4.get())) 
    cursor = mydb.cursor() 
    global init_table
    if((update_file == True) | (init_table == False)):
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            q = "DROP TABLE IF EXISTS "+str(entry5.get())
            cursor.execute(q)
        
        q = """CREATE TABLE IF NOT EXISTS """+ str(entry5.get())+"""(
       Name varchar(32) NOT NULL,
       Intro varchar(255) NOT NULL,
        Location varchar(32) NOT NULL,
        Job varchar(255) NOT NULL,
        About varchar(5000) NOT NULL,
        Education varchar(5000) NOT NULL,
        Skills varchar(5000) NOT NULL,
        URL varchar(255) NOT NULL
    )"""
        cursor.execute(q)
    
        xl_data = xlrd.open_workbook(path)
        sheet = xl_data.sheet_by_index(0)

        query = """INSERT INTO """+str(entry5.get())+""" (Name, Intro, Location, Job, About, Education, Skills, URL) 
                               VALUES (%s, %s, %s, %s, %s, %s, %s, %s) """

        for i in range(1,sheet.nrows):
            Name = sheet.cell(i,0).value
            Intro = sheet.cell(i,1).value
            Location = sheet.cell(i,2).value
            Job = sheet.cell(i,3).value
            About = sheet.cell(i,4).value
            Education = sheet.cell(i,5).value
            Skills = sheet.cell(i,6).value
            URL = sheet.cell(i,7).value

            values = (Name,Intro,Location,Job,About,Education,Skills,URL)

            cursor.execute(query,values)
        init_table = True
        label3 = tk.Label(root, text= 'Table Initialized!',font=('helvetica', 10))
        canvas1.create_window(600, 385, window=label3)
    else:
        label3 = tk.Label(root, text= 'Table Already Initialized!',font=('helvetica', 10))
        canvas1.create_window(600, 385, window=label3)
    cursor.close()
    mydb.commit()
    mydb.close()
#     label3 = tk.Label(root, text= 'Table Initialized!',font=('helvetica', 10))
#     canvas1.create_window(600, 390, window=label3)
    

In [8]:
def search(): 
    from tabulate import tabulate
    t = entry7.get()
    mydb = pymysql.connect( host = str(entry1.get()) ,  user =str(entry2.get()) ,  passwd = str(entry3.get()) , db = str(entry4.get()))  

    cursor = mydb.cursor() 
    
    e = str(entry5.get())
    #t = str(input("Enter a Text Query:"))
    f_at = t.find(' at ')
    f_in = t.find(' in ')

    if((f_at == -1) & (f_in==-1)):
        q = """SELECT * FROM """+e+""" WHERE Location LIKE "%"""+t+"""%"
       OR Job LIKE "%"""+t+"""%"
       OR Skills LIKE "%"""+t+"""%" """
        cursor.execute(q)
        res = cursor.fetchall()
        if(len(res) == 0):
            cursor.close()
            quer = """ALTER TABLE """+e+""" ADD FULLTEXT(Location, Job, Skills)"""
            query = """SELECT * FROM """+e+""" WHERE MATCH(Location, Job, Skills)
                    AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
            cursor = mydb.cursor() 
            cursor.execute(quer)
            cursor.execute(query)
            res = cursor.fetchall()




    elif((f_at != -1) & (f_in==-1)):
        before_at = t[0:f_at]
        after_at = t[f_at+4:]
        #print(before_at,after_at)
        q = """SELECT * FROM """+e+""" WHERE ((Job LIKE "%"""+t+"""%")
       OR ((Job LIKE "%"""+before_at+"""%"
       OR Skills LIKE "%"""+before_at+"""%")
       AND Location LIKE "%"""+after_at+"""%"))"""
        cursor.execute(q)
        res = cursor.fetchall()
        if(len(res) == 0):
            cursor.close()
            quer = """ALTER TABLE """+e+""" ADD FULLTEXT(Location, Job, Skills)"""
            query = """SELECT * FROM """+e+""" WHERE MATCH(Location, Job, Skills)
                    AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
            cursor = mydb.cursor() 
            cursor.execute(quer)
            cursor.execute(query)
            res = cursor.fetchall()


    elif((f_at == -1) & (f_in!=-1)):
        before_in = t[0:f_in]
        after_in = t[f_in+4:]
        #print(before_in,after_in)
        q = """SELECT * FROM """+e+""" WHERE ((Location LIKE '%"""+after_in+"""%')
       AND ((Job LIKE '%"""+before_in+"""%'
       OR Skills LIKE '%"""+before_in+"""%')))"""
        cursor.execute(q)
        res = cursor.fetchall()
        if(len(res) == 0):
            cursor.close()
            quer = """ALTER TABLE """+e+""" ADD FULLTEXT(Location, Job, Skills)"""
            query = """SELECT * FROM """+e+""" WHERE MATCH(Location, Job, Skills)
                    AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
            cursor = mydb.cursor() 
            cursor.execute(quer)
            cursor.execute(query)
            res = cursor.fetchall()


    else:
        between_at_in = t[f_at+4:f_in]
        after_in = t[f_in+4:]
        before_at = t[0:f_at]
        before_in = t[0:f_in]
        q = """SELECT * FROM """+e+""" WHERE (Location LIKE '%"""+after_in+"""%'
       AND ((Job LIKE '%"""+before_in+"""%'
       OR Skills LIKE '%"""+before_at+"""%')))"""
        cursor.execute(q)
        res = cursor.fetchall()
        if(len(res) == 0):
            cursor.close()
            quer = """ALTER TABLE """+e+""" ADD FULLTEXT(Location, Job, Skills)"""
            query = """SELECT * FROM """+e+""" WHERE MATCH(Location, Job, Skills)
                    AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
            cursor = mydb.cursor() 
            cursor.execute(quer)
            cursor.execute(query)
            res = cursor.fetchall()


    #print(str(len(res))+" relevant records found!\n")
    res = list(res)
    # k=0

    for i in range(len(res)):
        res[i] = list(res[i])
        del res[i][4 : 7]


    for r in res:
        #k=k+1
        #print("Record:"+str(k))
    #     print(r)
    #     print('\n')
        r[1] = r[1][0:10] + "[...]"
        r[2] = r[2][0:15] + "[...]"
        r[3] = r[3][8:19] + "[...]"

    res.insert(0, ['Name','Intro','Location', 'Job','URL'])
    #sqx_txt = Label(root, text = str(len(res))+" relevant records found!").pack(row=3, column=0)
    cursor.close()
    mydb.commit()
    mydb.close()
    
    l1 = tk.Label(text = str(len(res) - 1)+" relevant records found!\n")
    canvas1.create_window(600, 540, window=l1)
    class TabulateLabel(tk.Label):
    
        def __init__(self, parent, data, **kwargs):
            super().__init__(parent, 
                             font=('Consolas', 10), 
                             justify=tk.LEFT, anchor='nw', **kwargs)

            text = tabulate(data, headers='firstrow', tablefmt='psql', showindex=False)
            self.configure(text=text)


    class App(tk.Tk):
        def __init__(self):
            super().__init__()
            TabulateLabel(self, data=res, bg='white').grid(sticky='ew')

    if __name__ == "__main__":
        App().mainloop()


In [ ]:
# Tkinter

# 4 text boxes, host, user, password, db, table name
from tkinter import * 
import tkinter as tk
from tkinter.ttk import *
  
# importing askopenfile function 
# from class filedialog 
from tkinter.filedialog import askopenfile 
import pymysql
path = ""
update_file = False
init_table = False
root= tk.Tk()
root.title("Simple Job Search")
canvas1 = tk.Canvas(root, width = 1280, height = 720)
canvas1.pack()


label1 = tk.Label(root, text='Host Name:')
label1.config(font=('helvetica', 10))
canvas1.create_window(200, 100, window=label1)

label2 = tk.Label(root, text='User Name:')
label2.config(font=('helvetica', 10))
canvas1.create_window(400, 100, window=label2)

label3 = tk.Label(root, text='Password:')
label3.config(font=('helvetica', 10))
canvas1.create_window(600, 100, window=label3)

label4 = tk.Label(root, text='Database:')
label4.config(font=('helvetica', 10))
canvas1.create_window(800, 100, window=label4)

label5 = tk.Label(root, text='Table Name:')
label5.config(font=('helvetica', 10))
canvas1.create_window(1000, 100, window=label5)

entry1 = tk.Entry (root) 
canvas1.create_window(200, 120, window=entry1)

entry2 = tk.Entry (root) 
canvas1.create_window(400, 120, window=entry2)

entry3 = tk.Entry (root,show="*") 
canvas1.create_window(600, 120, window=entry3)

entry4 = tk.Entry (root) 
canvas1.create_window(800, 120, window=entry4)

entry5 = tk.Entry (root) 
canvas1.create_window(1000, 120, window=entry5)
    
button1 = tk.Button(text='Create Database', command=create_database, bg='brown', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(500, 180, window=button1)

button2 = tk.Button(text='Create Table', command=create_table, bg='brown', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(700, 180, window=button2)

button3 = tk.Button(text='Browse File', command=Load_Database, bg='brown', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(500, 250, window=button3)

entry6 = tk.Entry (root) 
canvas1.create_window(500, 280, window=entry6)

button4 = tk.Button(text='Update File', command=update_database, bg='brown', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(700, 250, window=button4)

button5 = tk.Button(text='Initialize Table', command=initialize_table, bg='brown', fg='white', font=('helvetica', 15, 'bold'))
canvas1.create_window(600, 350, window=button5)

label6 = tk.Label(root, text='Enter your Query:')
label6.config(font=('helvetica', 15))
canvas1.create_window(600, 420, window=label6)

entry7 = tk.Entry (root, width=50) 
canvas1.create_window(600, 450, window=entry7)

button6 = tk.Button(text='Search', command=search, bg='green', fg='white', font=('helvetica', 15, 'bold'))
canvas1.create_window(600, 500, window=button6)

button7 = tk.Button(text='Refresh Table', command=refresh, bg='orange', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(600, 600, window=button7)


root.mainloop()